In [1]:
# import libraries
import pandas as pd
import yfinance as yf
import hvplot.pandas
import numpy as np


import warnings
warnings.filterwarnings('ignore')

In [2]:
# download the historical prices of pltr
pltr_df = yf.download(tickers = 'pltr', period='2Y', interval = '1h')
pltr_df

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,PLTR,PLTR,PLTR,PLTR,PLTR
Datetime,,,,,
2023-06-26 13:30:00+00:00,14.285000,14.420000,13.830000,13.870000,16317070
2023-06-26 14:30:00+00:00,14.120000,14.317500,13.980000,14.287400,8203271
2023-06-26 15:30:00+00:00,13.890000,14.210000,13.880000,14.110000,6981340
2023-06-26 16:30:00+00:00,14.022000,14.030000,13.860000,13.890000,5040281
2023-06-26 17:30:00+00:00,14.090000,14.110000,14.000000,14.025000,3773132
...,...,...,...,...,...
2025-06-25 15:30:00+00:00,142.610703,143.830002,142.105606,143.805206,6562395
2025-06-25 16:30:00+00:00,142.830002,142.869995,141.529999,142.630005,6777212


In [3]:
# Remove multilevels of the column headers
pltr_df = pltr_df.droplevel(level = 1, axis = 1)

# Remove the name 'Price' from the headers
pltr_df.columns.name = None

pltr_df

,Close,High,Low,Open,Volume
Datetime,,,,,
2023-06-26 13:30:00+00:00,14.285000,14.420000,13.830000,13.870000,16317070
2023-06-26 14:30:00+00:00,14.120000,14.317500,13.980000,14.287400,8203271
2023-06-26 15:30:00+00:00,13.890000,14.210000,13.880000,14.110000,6981340
2023-06-26 16:30:00+00:00,14.022000,14.030000,13.860000,13.890000,5040281
2023-06-26 17:30:00+00:00,14.090000,14.110000,14.000000,14.025000,3773132
...,...,...,...,...,...
2025-06-25 15:30:00+00:00,142.610703,143.830002,142.105606,143.805206,6562395
2025-06-25 16:30:00+00:00,142.830002,142.869995,141.529999,142.630005,6777212
2025-06-25 17:30:00+00:00,142.729996,142.899994,142.149994,142.839996,3773889


In [4]:
# Round the values of the dataframe to 2 decimal points
signals_df = round(pltr_df[['Close', 'High', 'Low', 'Open', 'Volume']], 2)

# Display the data
signals_df

,Close,High,Low,Open,Volume
Datetime,,,,,
2023-06-26 13:30:00+00:00,14.28,14.42,13.83,13.87,16317070
2023-06-26 14:30:00+00:00,14.12,14.32,13.98,14.29,8203271
2023-06-26 15:30:00+00:00,13.89,14.21,13.88,14.11,6981340
2023-06-26 16:30:00+00:00,14.02,14.03,13.86,13.89,5040281
2023-06-26 17:30:00+00:00,14.09,14.11,14.00,14.02,3773132
...,...,...,...,...,...
2025-06-25 15:30:00+00:00,142.61,143.83,142.11,143.81,6562395
2025-06-25 16:30:00+00:00,142.83,142.87,141.53,142.63,6777212
2025-06-25 17:30:00+00:00,142.73,142.90,142.15,142.84,3773889


In [5]:
# visulise the closing price
signals_df['Close'].hvplot()

:Curve   [Datetime]   (Close)

In [6]:
# Set the long and short windows
short_window = 20
long_window = 50

# Obtain the Exponential Moving Average of the Close prices with short and long windows
signals_df['EMA20_Close'] =  round(signals_df['Close'].ewm(span = short_window).mean(), 2)
signals_df['EMA50_Close'] =  round(signals_df['Close'].ewm(span = long_window).mean(), 2)

# Calculate Relative Strength Index (RSI)
delta = signals_df['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
signals_df['RSI'] = 100 - (100 / (1 + rs))

# Calculate the 14-day Average True Range (ATR) for volatility
high_low = signals_df['High'] - signals_df['Low']
high_close = (signals_df['High'] - signals_df['Close'].shift()).abs()
low_close = (signals_df['Low'] - signals_df['Close'].shift()).abs()
tr = pd.concat([high_low, high_close, low_close], axis=1)
signals_df['ATR'] = tr.max(axis=1).rolling(window=14).mean()

# Drop missing values
signals_df.dropna(inplace=True)

# view data
signals_df.head()

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR
Datetime,,,,,,,,,
2023-06-27 19:30:00+00:00,14.61,14.74,14.61,14.73,5940177,14.40,14.35,60.000000,0.227143
2023-06-28 13:30:00+00:00,15.18,15.18,14.47,14.53,16993470,14.50,14.42,70.270270,0.235714
2023-06-28 14:30:00+00:00,15.11,15.43,15.03,15.18,15291320,14.57,14.48,73.239437,0.240000
2023-06-28 15:30:00+00:00,15.15,15.31,15.09,15.12,6474473,14.64,14.53,82.474227,0.232143
2023-06-28 16:30:00+00:00,15.04,15.15,14.90,15.15,7077440,14.68,14.57,76.562500,0.237857


In [7]:
# Create signals
signals_df['Price_Change'] = signals_df['EMA20_Close'] - signals_df['EMA50_Close']
signals_df['Target'] = (signals_df['Price_Change'] > 0).astype(int)

# Display data
signals_df.head()

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Price_Change,Target
Datetime,,,,,,,,,,,
2023-06-27 19:30:00+00:00,14.61,14.74,14.61,14.73,5940177,14.40,14.35,60.000000,0.227143,0.05,1
2023-06-28 13:30:00+00:00,15.18,15.18,14.47,14.53,16993470,14.50,14.42,70.270270,0.235714,0.08,1
2023-06-28 14:30:00+00:00,15.11,15.43,15.03,15.18,15291320,14.57,14.48,73.239437,0.240000,0.09,1
2023-06-28 15:30:00+00:00,15.15,15.31,15.09,15.12,6474473,14.64,14.53,82.474227,0.232143,0.11,1
2023-06-28 16:30:00+00:00,15.04,15.15,14.90,15.15,7077440,14.68,14.57,76.562500,0.237857,0.11,1


## Machine Learning

In [8]:
# Import dependencies for the data processing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Define the datasets X and y for machine learing
X = signals_df.drop(columns = ['Target'], axis = 1)
y = signals_df['Target']

# Split the data into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, shuffle = False)

In [9]:
# Scale the data using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Conduct PCA transformation
pca = PCA(n_components= 4)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [10]:
# Import dependencies for machine learning
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

# initiate the model
gb_model = GradientBoostingClassifier(n_estimators = 300, max_depth = 5, learning_rate = .2, random_state = 1)

# fit the model with the training data
gb_model.fit(X_train_pca, y_train)

# Obtain the prediction with the model
y_pred_gb = gb_model.predict(X_test_pca)


In [11]:
from collections import Counter
print(Counter(y_pred_gb))

Counter({1: 632, 0: 413})


In [12]:
print(classification_report(y_test, y_pred_gb))

              precision    recall  f1-score   support

           0       0.54      0.76      0.63       294
           1       0.89      0.75      0.81       751

    accuracy                           0.75      1045
   macro avg       0.71      0.75      0.72      1045
weighted avg       0.79      0.75      0.76      1045



In [13]:
# Import SVC
from sklearn.svm import SVC

# Initialize and fit the data to SVC
svm_model = SVC(kernel = 'linear', gamma = 'auto', C=1.0)
svm_model.fit(X_train_pca, y_train)

# make predictions with the model
y_pred_svm = svm_model.predict(X_test_pca)

# Obtain the classification report of predictions against the test data
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       1.00      0.22      0.36       294
           1       0.77      1.00      0.87       751

    accuracy                           0.78      1045
   macro avg       0.88      0.61      0.61      1045
weighted avg       0.83      0.78      0.73      1045



In [14]:
print(Counter(y_pred_svm))

Counter({1: 980, 0: 65})


In [15]:
# Import Random Forest model
from sklearn.ensemble import RandomForestClassifier

# Initialize and fit the data to RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators = 500, max_depth = 5, random_state = 2)
model_rf.fit(X_train_pca, y_train)

# Obtain the predictions with the model
y_pred_rf = model_rf.predict(X_test_pca)

# Generate the classification report of the predictions against the test set
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.54      0.67      0.60       294
           1       0.86      0.77      0.81       751

    accuracy                           0.74      1045
   macro avg       0.70      0.72      0.71      1045
weighted avg       0.77      0.74      0.75      1045



In [16]:
print(Counter(y_pred_rf))

Counter({1: 676, 0: 369})


In [17]:
# Import Random Forest model
from sklearn.ensemble import AdaBoostClassifier

# Initialize and fit the data to RandomForestClassifier
model_ada = AdaBoostClassifier(n_estimators = 250, learning_rate = 0.5,  random_state = 10)
model_ada.fit(X_train_pca, y_train)

# Obtain the predictions with the model
y_pred_ada = model_ada.predict(X_test_pca)

# Generate the classification report of the predictions against the test set
print(classification_report(y_test, y_pred_ada))

              precision    recall  f1-score   support

           0       0.50      0.88      0.64       294
           1       0.94      0.66      0.77       751

    accuracy                           0.72      1045
   macro avg       0.72      0.77      0.71      1045
weighted avg       0.81      0.72      0.73      1045



In [18]:
print(Counter(y_pred_ada))

Counter({1: 526, 0: 519})


## Backtesting

In [19]:
# Create a simple backtesting algorithm
initial_capital = 100000
capital = initial_capital
position = 0

portfolio = []


for i in range(len(X_test)):
    if y_pred_ada[i] == 1:

        if position == 0:
            position = capital / X_test.iloc[i]['Close']
            capital -= position * X_test.iloc[i]['Close']

    elif y_pred_ada[i] == 0 and position !=0:
        capital = capital + X_test.iloc[i]['Close'] * position
        position = 0
    
    portfolio_value = capital + position * X_test.iloc[i]['Close']
    portfolio.append(portfolio_value)

portfolio_df = pd.DataFrame(portfolio , columns = ['Portfolio_Total'], index = X_test.index)

portfolio_df['Model_Predictions'] = y_pred_ada

portfolio_df.head()

,Portfolio_Total,Model_Predictions
Datetime,,
2024-11-14 18:30:00+00:00,100000.000000,1
2024-11-14 19:30:00+00:00,99548.721377,1
2024-11-14 20:30:00+00:00,98930.302524,1
2024-11-15 14:30:00+00:00,105047.634966,1
2024-11-15 15:30:00+00:00,108574.293833,1


In [20]:
# Merge X_test with  y_test and portfolio_df
x_test_with_returns = pd.concat([X_test, y_test, portfolio_df], axis = 1)
x_test_with_returns

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Price_Change,Target,Portfolio_Total,Model_Predictions
Datetime,,,,,,,,,,,,,
2024-11-14 18:30:00+00:00,59.83,59.84,59.33,59.47,4138157,59.85,57.09,50.536353,1.095714,2.76,1,100000.000000,1
2024-11-14 19:30:00+00:00,59.56,59.87,59.26,59.82,4321193,59.83,57.18,48.842593,1.113571,2.65,1,99548.721377,1
2024-11-14 20:30:00+00:00,59.19,59.60,58.97,59.56,4233481,59.77,57.26,46.179775,1.140714,2.51,1,98930.302524,1
2024-11-15 14:30:00+00:00,62.85,63.50,60.91,61.01,43869465,60.06,57.48,49.737671,1.197143,2.58,1,105047.634966,1
2024-11-15 15:30:00+00:00,64.96,65.04,62.83,62.84,29848767,60.53,57.77,58.635579,1.292143,2.76,1,108574.293833,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-25 15:30:00+00:00,142.61,143.83,142.11,143.81,6562395,141.46,139.47,80.260047,1.707857,1.99,1,198760.332971,0
2025-06-25 16:30:00+00:00,142.83,142.87,141.53,142.63,6777212,141.59,139.60,75.111773,1.542143,1.99,1,198760.332971,0
2025-06-25 17:30:00+00:00,142.73,142.90,142.15,142.84,3773889,141.70,139.72,75.000000,1.510714,1.98,1,198760.332971,0


In [21]:
# Obtain the Daily returns of the portfolio and clean the data
x_test_with_returns['Portfolio_Returns'] = x_test_with_returns['Portfolio_Total'].pct_change()
x_test_with_returns.dropna(inplace = True)


x_test_with_returns

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Price_Change,Target,Portfolio_Total,Model_Predictions,Portfolio_Returns
Datetime,,,,,,,,,,,,,,
2024-11-14 19:30:00+00:00,59.56,59.87,59.26,59.82,4321193,59.83,57.18,48.842593,1.113571,2.65,1,99548.721377,1,-0.004513
2024-11-14 20:30:00+00:00,59.19,59.60,58.97,59.56,4233481,59.77,57.26,46.179775,1.140714,2.51,1,98930.302524,1,-0.006212
2024-11-15 14:30:00+00:00,62.85,63.50,60.91,61.01,43869465,60.06,57.48,49.737671,1.197143,2.58,1,105047.634966,1,0.061835
2024-11-15 15:30:00+00:00,64.96,65.04,62.83,62.84,29848767,60.53,57.77,58.635579,1.292143,2.76,1,108574.293833,1,0.033572
2024-11-15 16:30:00+00:00,63.39,65.32,62.97,64.96,20125815,60.80,57.99,54.890865,1.378571,2.81,1,105950.192211,1,-0.024169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-25 15:30:00+00:00,142.61,143.83,142.11,143.81,6562395,141.46,139.47,80.260047,1.707857,1.99,1,198760.332971,0,0.000000
2025-06-25 16:30:00+00:00,142.83,142.87,141.53,142.63,6777212,141.59,139.60,75.111773,1.542143,1.99,1,198760.332971,0,0.000000
2025-06-25 17:30:00+00:00,142.73,142.90,142.15,142.84,3773889,141.70,139.72,75.000000,1.510714,1.98,1,198760.332971,0,0.000000


In [22]:
x_test_with_returns['Cumulative_Returns'] = (1 + x_test_with_returns['Portfolio_Returns']).cumprod() - 1

x_test_with_returns

,Close,High,Low,Open,Volume,EMA20_Close,EMA50_Close,RSI,ATR,Price_Change,Target,Portfolio_Total,Model_Predictions,Portfolio_Returns,Cumulative_Returns
Datetime,,,,,,,,,,,,,,,
2024-11-14 19:30:00+00:00,59.56,59.87,59.26,59.82,4321193,59.83,57.18,48.842593,1.113571,2.65,1,99548.721377,1,-0.004513,-0.004513
2024-11-14 20:30:00+00:00,59.19,59.60,58.97,59.56,4233481,59.77,57.26,46.179775,1.140714,2.51,1,98930.302524,1,-0.006212,-0.010697
2024-11-15 14:30:00+00:00,62.85,63.50,60.91,61.01,43869465,60.06,57.48,49.737671,1.197143,2.58,1,105047.634966,1,0.061835,0.050476
2024-11-15 15:30:00+00:00,64.96,65.04,62.83,62.84,29848767,60.53,57.77,58.635579,1.292143,2.76,1,108574.293833,1,0.033572,0.085743
2024-11-15 16:30:00+00:00,63.39,65.32,62.97,64.96,20125815,60.80,57.99,54.890865,1.378571,2.81,1,105950.192211,1,-0.024169,0.059502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-25 15:30:00+00:00,142.61,143.83,142.11,143.81,6562395,141.46,139.47,80.260047,1.707857,1.99,1,198760.332971,0,0.000000,0.987603
2025-06-25 16:30:00+00:00,142.83,142.87,141.53,142.63,6777212,141.59,139.60,75.111773,1.542143,1.99,1,198760.332971,0,0.000000,0.987603
2025-06-25 17:30:00+00:00,142.73,142.90,142.15,142.84,3773889,141.70,139.72,75.000000,1.510714,1.98,1,198760.332971,0,0.000000,0.987603


In [23]:
x_test_with_returns['Cumulative_Returns'].hvplot()

:Curve   [Datetime]   (Cumulative_Returns)

In [24]:
# Exit enrty points
x_test_with_returns['Entry/Exit'] = x_test_with_returns['Model_Predictions'].diff()
x_test_with_returns['Entry/Exit'][0] = x_test_with_returns['Model_Predictions'][0]
x_test_with_returns['Entry/Exit'] = x_test_with_returns['Entry/Exit'].astype(int)
x_test_with_returns[['Model_Predictions', 'Entry/Exit']]
x_test_with_returns['Entry/Exit'].value_counts()

Entry/Exit
 0    996
 1     24
-1     24
Name: count, dtype: int64

In [25]:
# Plot the buys and sells on the graph
exit = x_test_with_returns[x_test_with_returns['Entry/Exit'] == -1]['Portfolio_Total'].hvplot.scatter(color = 'red',
                                       marker = 'v',
                                       legend = False,
                                       width = 1000,
                                       height = 500,
                                       size = 200, ylabel = 'Price in $')
exit

entry = x_test_with_returns[x_test_with_returns['Entry/Exit'] == 1]['Portfolio_Total'].hvplot.scatter(color = 'green',
                                       marker = '^',
                                       legend = False,
                                       width = 1000,
                                       height = 500,
                                       size = 200, ylabel = 'Price in $')

entry



portfolio_price_chart = x_test_with_returns['Portfolio_Total'].hvplot(color = 'lightgray')

entry_exit_chart = portfolio_price_chart * entry * exit 

entry_exit_chart.opts(title = 'Entry Exit Plot on the Portfolio Cumulative Returns of the model',
                     height = 500,
                     width = 1000)

:Overlay
   .Curve.Portfolio_Total      :Curve   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio_Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio_Total)

## Calculating Metrics

In [26]:
metics = ['Annualized Returns',
          'Cumulative Returns',
          'Annualized Volatility',
          'Sharpe Ratio',
          'Sortino Ratio']

evaluation_df = pd.DataFrame(columns = ['Backtest'], index = metics)
evaluation_df

,Backtest
Annualized Returns,NaN
Cumulative Returns,NaN
Annualized Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [27]:
# Add the first four respective data to the data frame
evaluation_df.loc['Annualized Returns'] = x_test_with_returns['Portfolio_Returns'].mean() * 252
evaluation_df.loc['Cumulative Returns'] = x_test_with_returns['Cumulative_Returns'][-1]
evaluation_df.loc['Annualized Volatility'] = x_test_with_returns['Portfolio_Returns'].std() * np.sqrt(252)
evaluation_df.loc['Sharpe Ratio'] = (x_test_with_returns['Portfolio_Returns'].mean() * 252) / (x_test_with_returns['Portfolio_Returns'].std() * np.sqrt(252))

# Display the data
evaluation_df

,Backtest
Annualized Returns,0.193894
Cumulative Returns,0.987603
Annualized Volatility,0.240183
Sharpe Ratio,0.807278
Sortino Ratio,NaN


In [28]:
# Convert the returns to numneric and drop 'NaN's for calulation
daily_returns = pd.to_numeric(x_test_with_returns['Portfolio_Returns']).dropna()


# Calculate average daily return
average_return = daily_returns.mean()

# Calculate downside returns (only negative returns)
downside_returns = daily_returns[daily_returns < 0]

# Calculate the downside standard deviation
downside_std = downside_returns.std()

# Calculate the Sortino Ratio
sortino_ratio = (average_return / downside_std) * np.sqrt(252)
sortino_ratio

0.7876943877721111

In [29]:
# Obtain the sortino ratio and add it to the evaluation dataframe
evaluation_df.loc['Sortino Ratio'] = sortino_ratio
evaluation_df

,Backtest
Annualized Returns,0.193894
Cumulative Returns,0.987603
Annualized Volatility,0.240183
Sharpe Ratio,0.807278
Sortino Ratio,0.787694


## Performance analysis of the strategy


In [30]:
# Create the dataframe specifying the features of the trades along with its profit and loss
performance_data = []

    
for index, row in x_test_with_returns.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_share_price = row['Close']
        share_size = abs(row['Portfolio_Total'] / row['Close'])
        entry_portfolio_holdings = row['Close'] * (abs(row['Portfolio_Total'] / row['Close']))

    elif row['Entry/Exit'] == -1 and entry_date is not None:
        exit_date = index
        exit_share_price = row['Close']
        share_size = abs(row['Portfolio_Total'] / row['Close'])
        exit_portfolio_holdings = row['Close'] * (abs(row['Portfolio_Total'] / row['Close']))
        profit_loss = exit_portfolio_holdings - entry_portfolio_holdings

        performance_data.append({
            'Stock': 'PLTR',
            'Entry Date': entry_date,
            'Exit Date': exit_date,
            'Entry Price': entry_share_price,
            'Exit Price': exit_share_price,
            'Shares': share_size,
            'Entry Portfolio Holding': entry_portfolio_holdings,
            'Exit Portfolio Holding': exit_portfolio_holdings,
            'Profit/Loss': profit_loss
        })

        

performance_data_df = pd.DataFrame(performance_data)

performance_data_df

,Stock,Entry Date,Exit Date,Entry Price,Exit Price,Shares,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,PLTR,2024-11-14 19:30:00+00:00,2024-11-21 14:30:00+00:00,59.56,61.64,1671.402307,99548.721377,103025.238175,3476.516798
1,PLTR,2024-11-21 15:30:00+00:00,2024-11-22 14:30:00+00:00,62.72,62.21,1642.621782,103025.238175,102187.501066,-837.737109
2,PLTR,2024-11-22 15:30:00+00:00,2024-11-27 14:30:00+00:00,63.34,66.31,1613.317036,102187.501066,106979.052663,4791.551597
3,PLTR,2024-11-27 15:30:00+00:00,2024-12-02 14:30:00+00:00,65.34,66.78,1637.267411,106979.052663,109336.717736,2357.665072
4,PLTR,2024-12-02 15:30:00+00:00,2024-12-03 14:30:00+00:00,66.84,69.85,1635.797692,109336.717736,114260.468789,4923.751053
5,PLTR,2024-12-03 15:30:00+00:00,2024-12-13 14:30:00+00:00,70.07,73.58,1630.661750,114260.468789,119984.091529,5723.622741
6,PLTR,2024-12-13 17:30:00+00:00,2024-12-27 18:30:00+00:00,74.22,78.79,1616.600533,119984.091529,127371.955963,7387.864434
7,PLTR,2025-01-21 15:30:00+00:00,2025-01-21 16:30:00+00:00,73.05,72.71,1743.627049,127371.955963,126779.122767,-592.833197
8,PLTR,2025-01-21 17:30:00+00:00,2025-01-21 20:30:00+00:00,73.28,73.09,1730.064448,126779.122767,126450.410521,-328.712245
9,PLTR,2025-01-22 16:30:00+00:00,2025-01-30 14:30:00+00:00,76.94,80.41,1643.493768,126450.410521,132153.333897,5702.923375
